In [41]:
# Dependencies
import json
import requests
import pprint
import pandas as pd
from NPS_config import api_key

In [60]:
# Define contants
NPS_BASE_URL = 'https://developer.nps.gov/api/v1/'
NPS_PARAMS = {
    "limit": 5000,
    "api_key": api_key,
    "parkCode": "acad,arch,badl,bibe,bisc,blca,brca,cany,care,cave,chis,cong,crla,cuva,deva,drto,ever,glac,grba,grca,grsm,grte,gumo,hale,havo,hosp,indu,isro,jeff,jotr,kefj,kova,lavo,maca,meve,mora,noca,olym,pefo,pinn,romo,sagu,shen,thro,viis,voya,whsa,wica,yell,yose,zion"
}

In [61]:
# Create get request to API
def nps_query(endpoint):
    url = NPS_BASE_URL + endpoint 
    response = requests.get(url, params=NPS_PARAMS)

    if response.ok:
        callback = response.json()
        if int(callback["total"]) >= int(callback["limit"]):
            raise ValueError(f'Error {url} records returned great than or equal to limit')   
    else:
        raise ValueError(f'Error {url} status code {response.status_code}')
    
    return callback

In [62]:
# Get unique API return for things to do at the national parks
nps_thingstodo_json = nps_query("thingstodo")

In [63]:
nps_thingstodo_json

{'total': '87',
 'limit': '5000',
 'start': '0',
 'data': [{'id': '2E20F5B9-D4DA-436B-8431-DC549C6B17E8',
   'url': 'https://www.nps.gov/thingstodo/swim-echo-lake-beach.htm',
   'title': 'Swim Echo Lake Beach',
   'shortDescription': 'Echo Lake Beach is a popular, freshwater swimming area on the southern end of Echo Lake. It is accessible off Route 102 in Southwest Harbor.',
   'images': [{'url': 'https://www.nps.gov/common/uploads/cropped_image/ADB236F7-DA77-A2E9-6DDC31B23E000D13.jpg',
     'credit': '',
     'altText': 'Child with an inflatable toy in a lake',
     'title': '',
     'description': '',
     'caption': '',
     'crops': [{'aspectRatio': '3',
       'url': 'https://www.nps.gov/common/uploads/cropped_image/primary/ADB236F7-DA77-A2E9-6DDC31B23E000D13.jpg'},
      {'aspectRatio': '1',
       'url': 'https://www.nps.gov/common/uploads/cropped_image/secondary/ADB236F7-DA77-A2E9-6DDC31B23E000D13.jpg'}]}],
   'relatedParks': [{'states': 'ME',
     'parkCode': 'acad',
     'des

In [67]:
# Initialize an empty list to store flattened data from the JSON response
thingstodo_data = nps_thingstodo_json.get('data', [])

# Flatten the nested JSON structure
flatten_ttd_data = []
for data in thingstodo_data:
    tags = data.get('tags', [])
    location = data.get('location', 'N/A')
    is_reservation_required = data.get('isReservationRequired', 'N/A')
    are_pets_permitted = data.get('arePetsPermitted', 'N/A')
    activities = data.get('activities', [])
    amenities = data.get('amenities', [])
    do_fees_apply = data.get('doFeesApply', 'N/A')
    season = data.get('season', [])
    duration = data.get('duration', 'N/A')
    
    related_parks = data.get('relatedParks', {})
    for related_park in related_parks:
        park_code = related_park.get('parkCode', 'N/A')
        designation = related_park.get('designation', 'N/A')
        full_name = related_park.get('fullName', 'N/A')
        states = related_park.get('states', 'N/A')
        
        flatten_ttd_data.append({
            'tags': tags,
            'location': location,
            'isReservationRequired': is_reservation_required,
            'arePetsPermitted': are_pets_permitted,
            'activities': activities,
            'amenities': amenities,
            'doFeesApply': do_fees_apply,
            'season': season,
            'duration': duration,
            'parkCode': park_code,
            'designation': designation,
            'fullName': full_name,
            'states': states
        })

# View the flattened things to do dataset
flatten_ttd_data

[{'tags': ['Acadia National Park',
   'Echo Lake',
   'echo lake beach',
   'west side',
   'swimming',
   'freshwater',
   'great ponds'],
  'location': 'Echo Lake Beach',
  'isReservationRequired': 'false',
  'arePetsPermitted': 'true',
  'activities': [{'id': '587BB2D3-EC35-41B2-B3F7-A39E2B088AEE',
    'name': 'Swimming'}],
  'amenities': [],
  'doFeesApply': 'false',
  'season': ['Summer'],
  'duration': '',
  'parkCode': 'acad',
  'designation': 'National Park',
  'fullName': 'Acadia National Park',
  'states': 'ME'},
 {'tags': ['stargazing',
   'night skies',
   'Acadia National Park',
   'cadillac mountain',
   'park loop road',
   'East Side',
   'mount desert island'],
  'location': 'Cadillac Mountain',
  'isReservationRequired': 'false',
  'arePetsPermitted': 'true',
  'activities': [{'id': 'D37A0003-8317-4F04-8FB0-4CF0A272E195',
    'name': 'Stargazing'}],
  'amenities': [],
  'doFeesApply': 'false',
  'season': ['Spring', 'Summer', 'Fall'],
  'duration': '',
  'parkCode': '

In [68]:
# Create a DataFrame from the extracted data
nps_ttd_data_df = pd.DataFrame(flatten_ttd_data)

# Display the DataFrame
nps_ttd_data_df.head(10)

,tags,location,isReservationRequired,arePetsPermitted,activities,amenities,doFeesApply,season,duration,parkCode,designation,fullName,states
0,"[Acadia National Park, Echo Lake, echo lake be...",Echo Lake Beach,false,true,[{'id': '587BB2D3-EC35-41B2-B3F7-A39E2B088AEE'...,[],false,[Summer],,acad,National Park,Acadia National Park,ME
1,"[stargazing, night skies, Acadia National Park...",Cadillac Mountain,false,true,[{'id': 'D37A0003-8317-4F04-8FB0-4CF0A272E195'...,[],false,"[Spring, Summer, Fall]",,acad,National Park,Acadia National Park,ME
2,[],Acadia Mountain Loop,false,true,[{'id': '45261C0A-00D8-4C27-A1F8-029F933A0D34'...,[],false,"[Spring, Summer, Fall]",2-3 Hours,acad,National Park,Acadia National Park,ME
3,[],Sargent and Penobscot Mountains from JPH,false,true,[{'id': '45261C0A-00D8-4C27-A1F8-029F933A0D34'...,[],false,"[Spring, Summer, Fall]",,acad,National Park,Acadia National Park,ME
4,[],Jordan Pond Loop,false,true,[{'id': '45261C0A-00D8-4C27-A1F8-029F933A0D34'...,[],false,"[Spring, Summer, Fall]",1-2 Hours,acad,National Park,Acadia National Park,ME
5,[],Sieur de Monts to Jesup Path and Hemlock Road ...,false,true,[{'id': '45261C0A-00D8-4C27-A1F8-029F933A0D34'...,[],false,"[Spring, Summer, Fall]",1-2 Hours,acad,National Park,Acadia National Park,ME
6,[],Sieur de Monts to Sand Beach,false,true,[{'id': '45261C0A-00D8-4C27-A1F8-029F933A0D34'...,[],false,"[Spring, Summer, Fall]",2-4 Hours,acad,National Park,Acadia National Park,ME
7,[],"Beech Cliff Trail, Beech Cliff Loop, and Canad...",false,false,[{'id': '45261C0A-00D8-4C27-A1F8-029F933A0D34'...,[],false,"[Spring, Summer, Fall]",2-4 Hours,acad,National Park,Acadia National Park,ME
8,[],Great Head Trail,false,true,[{'id': '45261C0A-00D8-4C27-A1F8-029F933A0D34'...,[],false,"[Spring, Summer, Fall]",1-2 Hours,acad,National Park,Acadia National Park,ME
9,[],The Ocean Path Trail,false,true,[{'id': '45261C0A-00D8-4C27-A1F8-029F933A0D34'...,[],false,"[Spring, Summer, Fall]",1-2 Hours,acad,National Park,Acadia National Park,ME


In [69]:
nps_ttd_data_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   tags                   87 non-null     object
 1   location               87 non-null     object
 2   isReservationRequired  87 non-null     object
 3   arePetsPermitted       87 non-null     object
 4   activities             87 non-null     object
 5   amenities              87 non-null     object
 6   doFeesApply            87 non-null     object
 7   season                 87 non-null     object
 8   duration               87 non-null     object
 9   parkCode               87 non-null     object
 10  designation            87 non-null     object
 11  fullName               87 non-null     object
 12  states                 87 non-null     object
dtypes: object(13)
memory usage: 9.0+ KB


In [70]:
# Export things to do data 

# Export DataFrame to a CSV file
nps_ttd_data_df.to_csv('nps_ttd_data.csv', index=False)
print("DataFrame exported to nps_ttd_data.csv")

# Export DataFrame to a JSON file
nps_ttd_data_df.to_json('nps_ttd_data.json', orient='records')
print("DataFrame exported to nps_ttd_data.json")

DataFrame exported to nps_ttd_data.csv
DataFrame exported to nps_ttd_data.json


In [49]:
# Get unique API return for amenitites at the national parks
nps_amen_place_json = nps_query("amenities/parksplaces")

In [50]:
# Get the data list from the JSON response

# Initialize an empty list to store flattened data from the JSON response
amenities_place_data = nps_amen_place_json.get('data', [])

# Flatten the nested JSON structure
flatten_ap_data = []
for data in amenities_place_data:
    for item in data:
        amenity_id = item.get('id', 'N/A')
        amenity_name = item.get('name', 'N/A')
        for park in item.get('parks', []):
            park_code = park.get('parkCode', 'N/A')
            park_name = park.get('fullName', 'N/A')
            park_location = park.get('states', 'N/A')
            park_designation = park.get('designation', 'N/A')
            park_url = park.get('url', 'N/A')
            flatten_ap_data.append({
                'amenity_id': amenity_id,
                'amenity_name': amenity_name,
                'park_code': park_code,
                'park_name': park_name,
                'park_states': park_location,
                'park_designation': park_designation,
                'park_url': park_url

            })

flatten_ap_data

[{'amenity_id': 'A1B0AD01-740C-41E7-8412-FBBEDD5F1443',
  'amenity_name': 'ATM/Cash Machine',
  'park_code': 'badl',
  'park_name': 'Badlands National Park',
  'park_states': 'SD',
  'park_designation': 'National Park',
  'park_url': 'http://www.nps.gov/badl/'},
 {'amenity_id': 'A1B0AD01-740C-41E7-8412-FBBEDD5F1443',
  'amenity_name': 'ATM/Cash Machine',
  'park_code': 'bibe',
  'park_name': 'Big Bend National Park',
  'park_states': 'TX',
  'park_designation': 'National Park',
  'park_url': 'http://www.nps.gov/bibe/'},
 {'amenity_id': 'A1B0AD01-740C-41E7-8412-FBBEDD5F1443',
  'amenity_name': 'ATM/Cash Machine',
  'park_code': 'brca',
  'park_name': 'Bryce Canyon National Park',
  'park_states': 'UT',
  'park_designation': 'National Park',
  'park_url': 'http://www.nps.gov/brca/'},
 {'amenity_id': 'A1B0AD01-740C-41E7-8412-FBBEDD5F1443',
  'amenity_name': 'ATM/Cash Machine',
  'park_code': 'cabr',
  'park_name': 'Cabrillo National Monument',
  'park_states': 'CA',
  'park_designation': 

In [51]:
# Create DataFrame from the flattened data
nps_amen_place_df = pd.DataFrame(flatten_ap_data)

# Display the DataFrame
nps_amen_place_df.head(10)

,amenity_id,amenity_name,park_code,park_name,park_states,park_designation,park_url
0,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,badl,Badlands National Park,SD,National Park,http://www.nps.gov/badl/
1,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,bibe,Big Bend National Park,TX,National Park,http://www.nps.gov/bibe/
2,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,brca,Bryce Canyon National Park,UT,National Park,http://www.nps.gov/brca/
3,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,cabr,Cabrillo National Monument,CA,National Monument,http://www.nps.gov/cabr/
4,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,deva,Death Valley National Park,"CA,NV",National Park,http://www.nps.gov/deva/
5,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,gate,Gateway National Recreation Area,"NY,NJ",National Recreation Area,http://www.nps.gov/gate/
6,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,glac,Glacier National Park,MT,National Park,http://www.nps.gov/glac/
7,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,grca,Grand Canyon National Park,AZ,National Park,http://www.nps.gov/grca/
8,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,gumo,Guadalupe Mountains National Park,TX,National Park,http://www.nps.gov/gumo/
9,A1B0AD01-740C-41E7-8412-FBBEDD5F1443,ATM/Cash Machine,hosp,Hot Springs National Park,AR,National Park,http://www.nps.gov/hosp/


In [58]:
nps_amen_place_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8652 entries, 0 to 8651
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   amenity_id        8652 non-null   object
 1   amenity_name      8652 non-null   object
 2   park_code         8652 non-null   object
 3   park_name         8652 non-null   object
 4   park_states       8652 non-null   object
 5   park_designation  8652 non-null   object
 6   park_url          8652 non-null   object
dtypes: object(7)
memory usage: 473.3+ KB


In [52]:
# Export ammenities and place data 

# Export DataFrame to a CSV file
nps_amen_place_df.to_csv('nps_amen_place_data.csv', index=False)
print("DataFrame exported to nps_amen_place_data.csv")

# Export DataFrame to a JSON file
nps_amen_place_df.to_json('nps_amen_place_data.json', orient='records')
print("DataFrame exported to nps_amen_place_data.json")

DataFrame exported to nps_amen_place_data.csv
DataFrame exported to nps_amen_place_data.json


In [53]:
# Get unique API return for the location information specific to the national parks
nps_parks_json = nps_query("parks")

In [54]:
# Extract and format the required information

# Initialize an empty list to store flattened data from the JSON response
nps_parks_data = nps_parks_json.get('data',[])

# Flatten the nested JSON structure
flatten_park_data = []
for data in nps_parks_data:
    park_id = data.get('id', 'N/A')
    park_url = data.get('url', 'N/A')
    park_name = data.get('fullName', 'N/A')
    park_code = data.get('parkCode', 'N/A')
    latitude = data.get('latitude', 'N/A')
    longitude = data.get('longitude', 'N/A')
    designation = data.get('designation', 'N/A')

    flatten_park_data.append({
        'park_id': park_id,
        'park_url': park_url,
        'park_name': park_name,
        'park_code': park_code,
        'park_latitude': latitude,
        'park_longitude': longitude,
        'park_designation': designation
    })

flatten_park_data

[{'park_id': '77E0D7F0-1942-494A-ACE2-9004D2BDC59E',
  'park_url': 'https://www.nps.gov/abli/index.htm',
  'park_name': 'Abraham Lincoln Birthplace National Historical Park',
  'park_code': 'abli',
  'park_latitude': '37.5858662',
  'park_longitude': '-85.67330523',
  'park_designation': 'National Historical Park'},
 {'park_id': '6DA17C86-088E-4B4D-B862-7C1BD5CF236B',
  'park_url': 'https://www.nps.gov/acad/index.htm',
  'park_name': 'Acadia National Park',
  'park_code': 'acad',
  'park_latitude': '44.409286',
  'park_longitude': '-68.247501',
  'park_designation': 'National Park'},
 {'park_id': 'E4C7784E-66A0-4D44-87D0-3E072F5FEF43',
  'park_url': 'https://www.nps.gov/adam/index.htm',
  'park_name': 'Adams National Historical Park',
  'park_code': 'adam',
  'park_latitude': '42.2553961',
  'park_longitude': '-71.01160356',
  'park_designation': 'National Historical Park'},
 {'park_id': '1A47416F-DAA3-4137-9F30-14AF86B4E547',
  'park_url': 'https://www.nps.gov/afam/index.htm',
  'park

In [55]:
# Create DataFrame from the flattened data
nps_parks_data_df = pd.DataFrame(flatten_park_data)

# Display the DataFrame
nps_parks_data_df.head(10)

,park_id,park_url,park_name,park_code,park_latitude,park_longitude,park_designation
0,77E0D7F0-1942-494A-ACE2-9004D2BDC59E,https://www.nps.gov/abli/index.htm,Abraham Lincoln Birthplace National Historical...,abli,37.5858662,-85.67330523,National Historical Park
1,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,https://www.nps.gov/acad/index.htm,Acadia National Park,acad,44.409286,-68.247501,National Park
2,E4C7784E-66A0-4D44-87D0-3E072F5FEF43,https://www.nps.gov/adam/index.htm,Adams National Historical Park,adam,42.2553961,-71.01160356,National Historical Park
3,1A47416F-DAA3-4137-9F30-14AF86B4E547,https://www.nps.gov/afam/index.htm,African American Civil War Memorial,afam,38.9166,-77.026,
4,E6E1D22A-7A89-47F8-813C-B611059A8CF9,https://www.nps.gov/afbg/index.htm,African Burial Ground National Monument,afbg,40.71452681,-74.00447358,National Monument
5,64B9F127-31ED-43C9-882D-D7CD471AF314,https://www.nps.gov/agfo/index.htm,Agate Fossil Beds National Monument,agfo,42.42170419,-103.753886,National Monument
6,3B8307B3-54AB-4E5F-ACBC-8DECB98AD5F1,https://www.nps.gov/alka/index.htm,Ala Kahakai National Historic Trail,alka,19.144668109,-155.890734294,National Historic Trail
7,C4E1A9A4-D121-4734-94FA-7788A93C2AAA,https://www.nps.gov/alag/index.htm,Alagnak Wild River,alag,59.05180188,-156.112002,Wild River
8,7D6098F9-3F75-4775-A56D-CD9C7F9A9488,https://www.nps.gov/anch/index.htm,Alaska Public Lands,anch,61.2188,-149.894536,
9,C08AD828-98FF-478E-A63C-614E7534274B,https://www.nps.gov/alca/index.htm,Alcatraz Island,alca,37.82676234,-122.4230206,


In [59]:
nps_parks_data_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   park_id           471 non-null    object
 1   park_url          471 non-null    object
 2   park_name         471 non-null    object
 3   park_code         471 non-null    object
 4   park_latitude     471 non-null    object
 5   park_longitude    471 non-null    object
 6   park_designation  471 non-null    object
dtypes: object(7)
memory usage: 25.9+ KB


In [56]:
# Export park data 

# Export DataFrame to a CSV file
nps_parks_data_df.to_csv('nps_parks_data.csv', index=False)
print("DataFrame exported to nps_parks_data.csv")

# Export DataFrame to a JSON file
nps_parks_data_df.to_json('nps_parks_data.json', orient='records')
print("DataFrame exported to nps_parks_data.json")

DataFrame exported to nps_parks_data.csv
DataFrame exported to nps_parks_data.json


In [71]:
import os
import shutil

# Define the folder name
folder_name = 'NPS_Project_Extracted_Data'

# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Define the file names and paths
files = {
    'nps_ttd_data.json': 'nps_ttd_data.json',
    'nps_amen_place_data.json': 'nps_amen_place_data.json',
    'nps_parks_data.json': 'nps_parks_data.json', 
    'nps_ttd_data.csv': 'nps_ttd_data.csv',
    'nps_amen_place_data.csv': 'nps_amen_place_data.csv',
    'nps_parks_data.csv': 'nps_parks_data.csv'
}

# Move each file into the folder
for target_name, source_path in files.items():
    if os.path.exists(source_path):
        shutil.move(source_path, os.path.join(folder_name, target_name))

# Verify the files have been moved
print(f"Files moved to {folder_name}: {os.listdir(folder_name)}")



Files moved to NPS_Project_Extracted_Data: ['nps_amen_place_data.csv', 'nps_amen_place_data.json', 'nps_parks_data.csv', 'nps_parks_data.json', 'nps_ttd_data.csv', 'nps_ttd_data.json']
